In [ ]:
!pip install mne
!pip install plotly
!pip install pymatreader 
!pip install librosa
!pip install yasa
!pip install protobuf==3.19
!pip install --upgrade pip
!pip uninstall tensorflow
!pip install tensorflow

In [ ]:
!pip install plotly==5.3.1

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import glob

from scipy.io import wavfile, savemat
from scipy.fft import fft, fftfreq
from scipy import signal
from scipy.fft import fftshift
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

from IPython.display import Audio, display
from IPython.display import clear_output

import librosa
import mne
import pymatreader
import yasa
import seaborn as sns
import pyxdf

from mne.time_frequency import psd_array_multitaper
from scipy.integrate import simps
from yasa import sliding_window

import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# CNN packages
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD,RMSprop,Adam
from keras.utils import np_utils
from keras import regularizers

In [ ]:
# Set gpu as backend
import tensorflow as tf
import keras
config = tf.compat.v1.ConfigProto()
sess = tf.compat.v1.Session(config=config)
keras.backend.set_session(sess)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

# Importing preprocessed data

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Preprocessedv5/")
os.getcwd()

In [ ]:
files = glob.glob("*.set")
files

In [ ]:
def get_trials():
    trial_ctr_dict = {}
    for x in events[1].keys():
        if "ActionBeg" in x:
            trial_ctr_dict[x[10:]] = 0
    return trial_ctr_dict

In [ ]:
def get_event_info(marker_stream):
    # Get audio info
    event_time = {"Name": [], "StartTime": [], "EndTime": [], "handPos": [], "Hand": [], "Action": []}
    for idx in range(len(marker_stream[0])):
        marker_list = marker_stream[0][idx][2]
        marker = list(marker_stream[1].keys())[list(marker_stream[1].values()).index(marker_list)]
        if "ActionBeg" in marker:
            start_time = marker_stream[0][idx][0]
            end_time = marker_stream[0][idx+1][0]
            action_name = marker[10:]
            event_time["Name"].append(action_name)
            event_time["StartTime"].append(start_time)
            event_time["EndTime"].append(end_time)
            event_time["handPos"].append(action_name.split("-")[0])
            event_time["Hand"].append(action_name.split("-")[1])
            event_time["Action"].append(action_name.split("-")[2])
    return event_time

In [ ]:
def save_time_series_as_df(event_time, eeg_data, file_name):
    eeg_df = []
    channels = np.array([[x] for x in arr.ch_names])
    trial_ctr_dict = get_trials()
    for ts in range(len(event_time["Name"])):
        p_eeg  = eeg_data.iloc[event_time["StartTime"][ts] - 25: event_time["StartTime"][ts] + 500, 1:].T
        handPos = np.array([[event_time["handPos"][ts]]] * 64)
        Hand = np.array([[event_time["Hand"][ts]]] * 64)
        Action = np.array([[event_time["Action"][ts]]] * 64)
        Subject = np.array([[file_name[:-4]]] * 64)
        trial_type = event_time["handPos"][ts] + "-" + event_time["Hand"][ts] + "-" + event_time["Action"][ts]
        trial_ctr_dict[trial_type] += 1
        Trial = np.array([[trial_ctr_dict[trial_type]]] * 64)

        p_eeg = np.append(p_eeg, handPos, axis=1)
        p_eeg = np.append(p_eeg, Hand, axis=1)
        p_eeg = np.append(p_eeg, Action, axis=1)
        p_eeg = np.append(p_eeg, Subject, axis=1)
        p_eeg = np.append(p_eeg, channels, axis=1)
        p_eeg = np.append(p_eeg, Trial, axis=1)

        if len(eeg_df) == 0:
            eeg_df = p_eeg
        else: 
            try:
                eeg_df = np.concatenate((eeg_df, p_eeg), axis=0)
            except:
                pass
        print(eeg_df.shape)

    return eeg_df 

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Preprocessedv5/")
print("Current directory: ", os.getcwd())
data_files = glob.glob("*.set")
print("Files in current directory: ", data_files)
eeg_df_main = []
for file_name in data_files:
    print("Current file: ", file_name)
    arr = mne.io.read_raw_eeglab(file_name)
    eeg_data = arr.to_data_frame()
    events = mne.events_from_annotations(arr)
    event_time = get_event_info(events)
    event_time
    eeg_df = save_time_series_as_df(event_time, eeg_data, file_name)
    
    if len(eeg_df_main) == 0:
        eeg_df_main = eeg_df
    else: 
        eeg_df_main = np.concatenate((eeg_df_main, eeg_df), axis=0)
    print(eeg_df_main.shape)

In [ ]:
df = pd.DataFrame(eeg_df_main, columns=list(range(-100, 2000, 4))+["handPos", "Hand", "Action", "Subject", "Channel", "Trial"])
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
df.to_csv("cleaned_datav5.csv", index=False, header=list(range(-100, 2000, 4))+["handPos", "Hand", "Action", "Subject", "Channel", "Trial"])
df

# Importing Data

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_datav3.csv")
dataset

In [ ]:
dataset[~(dataset["Subject"] == "P01")]

In [ ]:
dataset[(dataset["Subject"] == "P01")]

In [ ]:
# 64 channel EEG data with 500 samples
# Giving us X = N * 64 * 275 and y = N 
X, y = [], []
nchan = 64
new_dataset = dataset[~(dataset["Subject"] == "P01")]
iterations = len(new_dataset) // nchan
for idx in range(iterations):
    start = idx * nchan
    end = (idx + 1) * nchan

    X.append(np.expand_dims(new_dataset.iloc[start: end, :-6], 2))
    y.append(new_dataset.iloc[start, -4])
#     print(len(X), len(y))
X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
print(y)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
yl = le.transform(y)
yl

# ERP Analysis

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_datav2.csv")
dataset

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Preprocessedv5/")
arr = mne.io.read_raw_eeglab("P01.set")
#     eeg_data = arr.to_data_frame()
channels = arr.ch_names#np.array([[x] for x in ])
events = mne.events_from_annotations(arr)
channels

In [ ]:
trial_ctr_dict = {}
for x in events[1].keys():
    if "ActionBeg" in x:
        trial_ctr_dict[x[10:]] = []
trial_ctr_dict

In [ ]:
# Subject P02
for palm in ["palmDown", "palmIn", "palmUp"]:
    for oc in ["Open", "Close"]:
        for hand in ["Right"]:
            for i in range(40):
                test_sub_trial = dataset[
                    (dataset["handPos"] == palm) & 
                    (dataset["Action"] == oc) & 
                    (dataset["Hand"] == hand) & 
                    (dataset["Trial"] == i+1) & 
                    (dataset["Subject"] == "P03")]
                test_sub_trial.index = test_sub_trial.iloc[:, -2]
                name = palm + "-" + hand + "-" + oc 
                if len(trial_ctr_dict[name]) == 0:
                    trial_ctr_dict[name] = test_sub_trial.iloc[:, :-6].to_numpy()
                else:
                    trial_ctr_dict[name] += test_sub_trial.iloc[:, :-6].to_numpy()
                (name, test_sub_trial.shape, test_sub_trial)

In [ ]:
trial_ctr_dict_avg = {}
for key in trial_ctr_dict.keys():
    try:
        trial_ctr_dict_avg[key] = trial_ctr_dict[key] / 40
    except:
        pass
trial_ctr_dict_avg

In [ ]:
trial_ctr_dict

In [ ]:
exclusions = [0, 4, 16]
# channels = [i if i not in exclusions else -1 for i in list(range(64))]
chans = [3, 13, 20, 40, 56]
for key in trial_ctr_dict_avg.keys():
    fig = px.line(
        trial_ctr_dict_avg[key][:, :200].T, 
        title=key, 
    )
    fig.show()

In [ ]:
channels = [6]
# plt_1 = plt.figure(figsize=(10, 8))
# for channels in range(64):
for key in trial_ctr_dict_avg.keys():
    if "Open" in key:
        plt.plot(trial_ctr_dict_avg[key][channels, :150].T, "b")
    elif "Close" in key:
        plt.plot(trial_ctr_dict_avg[key][channels, :150].T, "g")

# plt.legend([ "Open", "Close"])
plt.title(channels)
plt.show()

# Visualizing Data

In [ ]:
test_sub_trial = dataset[(dataset["handPos"] == "palmDown") & (dataset["Action"] == "Open") & (dataset["Hand"] == "Right") & (dataset["Trial"] == 1) & (dataset["Subject"] == "P01")]
test_sub_trial

In [ ]:
os.getcwd()

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Plots/RawV3/")
for sub in ["P01", "P02", "P03", "P04"]:
    for palm in ["palmDown", "palmIn", "palmUp"]:
        for oc in ["Open", "Close"]:
            for hand in ["Left", "Right"]:
                for i in range(40):
                    test_sub_trial = dataset[
                        (dataset["handPos"] == palm) & 
                        (dataset["Action"] == oc) & 
                        (dataset["Hand"] == hand) & 
                        (dataset["Trial"] == i+1) & 
                        (dataset["Subject"] == sub)]
                    test_sub_trial.index = test_sub_trial.iloc[:, -2]
                    fig = px.line(
                        test_sub_trial.iloc[:, :-6].T, 
                        title=str(sub + palm + oc + hand + str(i)), 
                        labels=test_sub_trial.iloc[:, -2]
                    )
                    fig.write_image(sub + palm + oc + hand + str(i)+ ".png") 
                    fig.show()

In [ ]:
chanloc = "/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/EGIAmpServer.loc"
chans = mne.channels.read_custom_montage(chanloc)
chans.plot()
chans

In [ ]:
chanPos = chans.get_positions()
chanPos

In [ ]:
chanPos["ch_pos"]

In [ ]:
coords = []
for pos in chanPos["ch_pos"]:
    if "E" in pos:
        coords.append(np.array(chanPos["ch_pos"][pos][:2]))

coords = np.array(coords)
coords.shape

In [ ]:
channel_labels = list(chanPos["ch_pos"].keys())
print(channel_labels)

In [ ]:
plt.ion()
timestamps = list(range(-100, 1000, 4))
for time_step in range(275):
    print(timestamps[time_step])
    plt.title("Timestamp: " + str(timestamps[time_step]), fontsize=16)
    plt.rcParams["figure.figsize"] = (8,8)
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")

    mne.viz.plot_topomap(
      test_sub_trial.to_numpy()[:, time_step],
      coords, 
      # sensors="r.", 
      res=512, 
      names=channel_labels, 
      show_names=False, 
      outlines="skirt", 
    )

    plt.show()
    clear_output(wait=True)
    time.sleep(2)


In [ ]:
mne.viz.plot_topomap(
      test_sub_trial.to_numpy()[:, 7],
      coords, 
      # sensors="r.", 
      res=512, 
      names=channel_labels, 
      show_names=False, 
      outlines="skirt", 
    )

In [ ]:
test_sub_trial.to_numpy()[:, :-6].shape

# Classification

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_datav2_1.csv")
dataset

In [ ]:
# 64 channel EEG data with 500 samples
# Giving us X = N * 64 * 275 and y = N 
X, y = [], []
nchan = 64
new_dataset = dataset[~(dataset["Subject"] == "P01")]
iterations = len(new_dataset) // nchan
for idx in range(iterations):
    start = idx * nchan
    end = (idx + 1) * nchan

    X.append(np.expand_dims(new_dataset.iloc[start: end, :-6], 2))
    y.append(new_dataset.iloc[start, -4])
#     print(len(X), len(y))
X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
yl = le.transform(y)
yl

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X[:, :, 60: 124, 0], yl, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, X_test.shape, y_train.shape, y_eval.shape, y_test.shape

In [ ]:
#https://github.com/vlawhern/arl-eegmodels
def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.8, kernLength = 125, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(1, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('sigmoid', name = 'sigmoid')(dense)
    
    return Model(inputs=input1, outputs=softmax)

model = EEGNet(1, 64, 64)
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

# count number of parameters in the model
numParams = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpoint_filepath = "/tmp/checkpoint/"
checkpointer = ModelCheckpoint(
        filepath=checkpoint_filepath, 
        save_weights_only=True,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True
)

# Handle class imbalance
class_weights = {0:1, 1:1}

In [ ]:
################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################
batch_size = 512
EPOCHS = 500
history = model.fit(
    X_train, 
    y_train, 
    batch_size = batch_size, 
    epochs = EPOCHS, 
    validation_data=(X_eval, y_eval),
    callbacks=[checkpointer], 
#     class_weight = class_weights
)

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.show()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()

In [ ]:
# load optimal weights
model.load_weights(checkpoint_filepath)
path_name = '/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Models/model-eegnet-sig-crr-v3'
model.save(path_name)

In [ ]:
new_model = tf.keras.models.load_model(path_name)
# y_pred = np.argmax(new_model.predict(X_test), axis=1)  # Softmax
y_pred = [1 if x > 0.5 else 0 for x in new_model.predict(X_test)]  # Sigmoid
y_true = y_test
test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy main: {test_acc:.0%}')

In [ ]:
confusion_matrix = tf.math.confusion_matrix(y_true, y_pred)

plt.figure(figsize = (7,7))
hm1 = sns.heatmap(confusion_matrix, annot=True, cbar=False, vmin=0, vmax=1500,fmt='g')

hm1.set(title='True Action', ylabel='Predicted Action')
hm1.xaxis.tick_top()

hm1.set_xticklabels([int(i.get_text())+1 for i in hm1.get_xticklabels()])
hm1.set_yticklabels([int(i.get_text())+1 for i in hm1.get_yticklabels()])
plt.tight_layout()
# file_name = "beat_classification_confusion_matrix_reverse_sizes"
# os.chdir('/mnt/sda1/shivam/DrummingEEG/FullStudyData/images')
# hm1.get_figure().savefig(file_name + ".png")

# Using power spectral density

In [ ]:
X.shape

In [ ]:
# Multitaper method
def bandpower_multitaper(data, sf, method, band, relative=False):
    band = np.asarray(band)
    low, high = band

    if method == 'multitaper':
        psd_trial, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                                normalization='full', verbose=0)
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd_trial[idx_band], dx=freq_res)

    return bp


In [ ]:
def create_bandpower_features(epochs, frequency):
    bandpower_multitaper_EEG = []
    # Iterating over each subject [20]
    for epoch in epochs:
        # Iterating over each song per subject [30]
        bands_video = []
        no_channels = epoch.shape[0]
        input_brainwaves = epoch
        # Iterating over each channel [14]
        for k in range(no_channels):
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[0.5, 4], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[4, 7], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[8, 13], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[14, 30], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[31, 50], relative=False))
      
        bandpower_multitaper_EEG.append(bands_video)

    bandpower_multitaper_EEG = np.array(bandpower_multitaper_EEG)
  
    return bandpower_multitaper_EEG

In [ ]:
sf = 250
window = 2
psd_features = []
for x in X:
    epoch = sliding_window(x[:,:,0], sf=sf, window=window, step=window)[1]
    psd = create_bandpower_features(epoch, sf)
    if len(psd_features) == 0:
        psd_features = psd
    else:
        psd_features = np.concatenate((psd_features, psd))
    print(psd_features.shape)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data')
np.save("IntData/psd_features_v6.npy", psd_features)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
psd_labels = []
for y in yl:
    psd_labels += [y] * 1
np.save("psd_labels_v6.npy", psd_labels)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
X, y = np.load("psd_features_v6.npy"), np.load("psd_labels_v6.npy")
X.shape, y.shape # ((1920, 320), (1920,))

In [ ]:
X[y==1].shape

In [ ]:
# Common Spatial Pattern implementation in Python, used to build spatial filters for identifying task-related activity.
import numpy as np
import scipy.linalg as la

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
    if len(tasks) < 2:
        print("Must have at least 2 tasks for filtering.")
        return (None,) * len(tasks)
    else:
        filters = ()
        # CSP algorithm
        # For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
        iterator = range(0,len(tasks))
        for x in iterator:
            # Find Rx
            Rx = covarianceMatrix(tasks[x][0])
            for t in range(1,len(tasks[x])):
                Rx += covarianceMatrix(tasks[x][t])
            Rx = Rx / len(tasks[x])

            # Find not_Rx
            count = 0
            not_Rx = Rx * 0
            for not_x in [element for element in iterator if element != x]:
                for t in range(0,len(tasks[not_x])):
                    not_Rx += covarianceMatrix(tasks[not_x][t])
                    count += 1
            not_Rx = not_Rx / count

            # Find the spatial filter SFx
            SFx = spatialFilter(Rx,not_Rx)
            filters += (SFx,)

            # Special case: only two tasks, no need to compute any more mean variances
            if len(tasks) == 2:
                filters += (spatialFilter(not_Rx,Rx),)
                break
        return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
    Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
    return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
    R = Ra + Rb
    E,U = la.eig(R)

    # CSP requires the eigenvalues E and eigenvector U be sorted in descending order
    ord = np.argsort(E)
    ord = ord[::-1] # argsort gives ascending order, flip to get descending
    E = E[ord]
    U = U[:,ord]

    # Find the whitening transformation matrix
    P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

    # The mean covariance matrices may now be transformed
    Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
    Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

    # Find and sort the generalized eigenvalues and eigenvector
    E1,U1 = la.eig(Sa,Sb)
    ord1 = np.argsort(E1)
    ord1 = ord1[::-1]
    E1 = E1[ord1]
    U1 = U1[:,ord1]

    # The projection matrix (the spatial filter) may now be obtained
    SFa = np.dot(np.transpose(U1),P)
    return SFa.astype(np.float32)

In [ ]:
CSP(X[y==0], X[y==1])

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X[480:, :], y[480:], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train.shape, X_eval.shape, X_test.shape, y_train.shape, y_eval.shape, y_test.shape

In [ ]:
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

In [ ]:
# source : https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

clfs = [
    KNeighborsClassifier(2),
    # SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=200),
    RandomForestClassifier(max_depth=200, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

names = [
    "Nearest Neighbors",
    # "Linear SVM",
    "RBF SVM",
    # "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

clfs_mlp = [
    MLPClassifier(hidden_layer_sizes=(10, 20, 10), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(20, 40, 30), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(30, 60, 45), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(40, 80, 60), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(60, 120, 80), alpha=0.01, max_iter=1000),
    MLPClassifier(hidden_layer_sizes=(80, 140, 100), alpha=0.01, max_iter=1000),
]

names_mlp = [
    "MLP1",
    "MLP2",
    "MLP3",
    "MLP4",
    "MLP5",
    "MLP6",
]

In [ ]:
chanloc = "/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/EGIAmpServer.loc"
chans = mne.channels.read_custom_montage(chanloc)
chans.plot()
chans

In [ ]:
results = []
for idx,clf in enumerate(clfs):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    results.append([names[idx], score])
    print(names[idx], score)

In [ ]:
X_train[:, 2].shape

In [ ]:
D

In [ ]:
Trial = np.array([[trial_ctr_dict[trial_type]]] * 64)
p_eeg = np.append(p_eeg, handPos, axis=1)

In [ ]:
# Based on channel selection, 8 channels at a time. Then we go for iteratively increasing channels.
chan_sel_results = []
# clf = KNeighborsClassifier(2)
chunk = 64
chan_sel_results = np.array([[x] for x in names])
for i in range(64//chunk):
    start = i * chunk * 5
    end = (i+1) * chunk * 5
    score_chunk = []
    for idx,clf in enumerate(clfs):
        clf.fit(X_train[:, start:end], y_train)
        score = clf.score(X_test[:, start:end], y_test)
        score_chunk.append(score)
    score_chunk = np.array([[x] for x in score_chunk])
    chan_sel_results = np.append(chan_sel_results, score_chunk, axis=1)
chan_sel_results.shape, chan_sel_results

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results")
df = pd.DataFrame(data=chan_sel_results, columns=["classifier"] + ["Chunk" + str(i) for i in range(64//chunk)])
df.to_csv("chan_sel_results_64.csv", index=None)
df

In [ ]:
# df =  pd.DataFrame(data = chan_sel_results, columns =['Start','End', "Classifier", 'Testing Score'])
# os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Results")
# df.to_csv("chan_sel_results_4.csv", index=None)
# df

In [ ]:
y_new = y[480:]

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
perplexity = range(0, 20, 4)
for per in perplexity:
    dataset_embedded = TSNE(
        n_components=2, 
        learning_rate="auto", 
        init="random", 
        perplexity=per
    ).fit_transform(X[480:])
    
    print("Perplexity: ", per)
    plt.scatter(dataset_embedded[y_new==0,0], dataset_embedded[y_new==0,1])
    plt.scatter(dataset_embedded[y_new==1,0], dataset_embedded[y_new==1,1])
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

def get_signal_Hz(Hz,sample_rate,length_ts_sec):
    ## 1 sec length time series with sampling rate 
    ts1sec = list(np.linspace(0,np.pi*2*Hz,sample_rate))
    ## 1 sec length time series with sampling rate 
    ts = ts1sec*length_ts_sec
    return(list(np.sin(ts)))

sample_rate   = 4000
length_ts_sec = 3
## --------------------------------- ##
## 3 seconds of "digit 1" sound
## Pressing digit 2 buttom generates 
## the sine waves at frequency 
## 697Hz and 1209Hz.
## --------------------------------- ##
ts1  = np.array(get_signal_Hz(697, sample_rate,length_ts_sec)) 
ts1 += np.array(get_signal_Hz(1209,sample_rate,length_ts_sec))
ts1  = list(ts1)

## -------------------- ##
## 2 seconds of silence
## -------------------- ##
ts_silence = [0]*sample_rate*1

## --------------------------------- ##
## 3 seconds of "digit 2" sounds 
## Pressing digit 2 buttom generates 
## the sine waves at frequency 
## 697Hz and 1336Hz.
## --------------------------------- ##
ts2  = np.array(get_signal_Hz(697, sample_rate,length_ts_sec)) 
ts2 += np.array(get_signal_Hz(1336,sample_rate,length_ts_sec))
ts2  = list(ts2)

## -------------------- ##
## Add up to 7 seconds
## ------------------- ##
ts = ts1 + ts_silence  + ts2

In [ ]:


total_ts_sec = len(ts)/sample_rate
print("The total time series length = {} sec (N points = {}) ".format(total_ts_sec, len(ts)))
plt.figure(figsize=(20,3))
plt.plot(ts)
plt.xticks(np.arange(0,len(ts),sample_rate),
           np.arange(0,len(ts)/sample_rate,1))
plt.ylabel("Amplitude")
plt.xlabel("Time (second)")
plt.title("The total length of time series = {} sec, sample_rate = {}".format(len(ts)/sample_rate, sample_rate))
plt.show()



In [ ]:
def get_xn(Xs,n):
    '''
    calculate the Fourier coefficient X_n of 
    Discrete Fourier Transform (DFT)
    '''
    L  = len(Xs)
    ks = np.arange(0,L,1)
    xn = np.sum(Xs*np.exp((1j*2*np.pi*ks*n)/L))/L
    return(xn)

def get_xns(ts):
    '''
    Compute Fourier coefficients only up to the Nyquest Limit Xn, n=1,...,L/2
    and multiply the absolute value of the Fourier coefficients by 2, 
    to account for the symetry of the Fourier coefficients above the Nyquest Limit. 
    '''
    mag = []
    L = len(ts)
    for n in range(int(L/2)): # Nyquest Limit
        mag.append(np.abs(get_xn(ts,n))*2)
    return(mag)
mag = get_xns(ts)

In [ ]:
# the number of points to label along xaxis
Nxlim = 10

plt.figure(figsize=(20,3))
plt.plot(mag)
plt.xlabel("Frequency (k)")
plt.title("Two-sided frequency plot")
plt.ylabel("|Fourier Coefficient|")
plt.show()

In [ ]:
def get_Hz_scale_vec(ks,sample_rate,Npoints):
    freq_Hz = ks*sample_rate/Npoints
    freq_Hz  = [int(i) for i in freq_Hz ] 
    return(freq_Hz )

ks   = np.linspace(0,len(mag),Nxlim)
ksHz = get_Hz_scale_vec(ks,sample_rate,len(ts))

plt.figure(figsize=(20,3))
plt.plot(mag)
plt.xticks(ks,ksHz)
plt.title("Frequency Domain")
plt.xlabel("Frequency (Hz)")
plt.ylabel("|Fourier Coefficient|")
plt.show()

In [ ]:
def create_spectrogram(ts,NFFT,noverlap = None):
    '''
          ts: original time series
        NFFT: The number of data points used in each block for the DFT.
          Fs: the number of points sampled per second, so called sample_rate
    noverlap: The number of points of overlap between blocks. The default value is 128. 
    '''
    if noverlap is None:
        noverlap = NFFT/2
    noverlap = int(noverlap)
    starts  = np.arange(0,len(ts),NFFT-noverlap,dtype=int)
    # remove any window with less than NFFT sample size
    starts  = starts[starts + NFFT < len(ts)]
    xns = []
    for start in starts:
        # short term discrete fourier transform
        ts_window = get_xns(ts[start:start + NFFT]) 
        xns.append(ts_window)
    specX = np.array(xns).T
    # rescale the absolute value of the spectrogram as rescaling is standard
    spec = 10*np.log10(specX)
    assert spec.shape[1] == len(starts) 
    return(starts,spec)

L = 256
noverlap = 84
starts, spec = create_spectrogram(ts,L,noverlap = noverlap )



In [ ]:
def plot_spectrogram(spec,ks,sample_rate, L, starts, mappable = None):
    plt.figure(figsize=(20,8))
    plt_spec = plt.imshow(spec,origin='lower')

    ## create ylim
    Nyticks = 10
    ks      = np.linspace(0,spec.shape[0],Nyticks)
    ksHz    = get_Hz_scale_vec(ks,sample_rate,len(ts))
    plt.yticks(ks,ksHz)
    plt.ylabel("Frequency (Hz)")

    ## create xlim
    Nxticks = 10
    ts_spec = np.linspace(0,spec.shape[1],Nxticks)
    ts_spec_sec  = ["{:4.2f}".format(i) for i in np.linspace(0,total_ts_sec*starts[-1]/len(ts),Nxticks)]
    plt.xticks(ts_spec,ts_spec_sec)
    plt.xlabel("Time (sec)")

    plt.title("Spectrogram L={} Spectrogram.shape={}".format(L,spec.shape))
    plt.colorbar(mappable,use_gridspec=True)
    plt.show()
    return(plt_spec)
plot_spectrogram(spec,ks,sample_rate,L, starts)



In [ ]:
# -*- coding: utf-8 -*-
"""
.. _ex-decoding-csp-eeg:

===========================================================================
Motor imagery decoding from EEG data using the Common Spatial Pattern (CSP)
===========================================================================

Decoding of motor imagery applied to EEG data decomposed using CSP. A
classifier is then applied to features extracted on CSP-filtered signals.

See https://en.wikipedia.org/wiki/Common_spatial_pattern and
:footcite:`Koles1991`. The EEGBCI dataset is documented in
:footcite:`SchalkEtAl2004` and is available at PhysioNet
:footcite:`GoldbergerEtAl2000`.
"""
# Authors: Martin Billinger <martin.billinger@tugraz.at>
#
# License: BSD-3-Clause

# %%


import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

print(__doc__)

# #############################################################################
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.
tmin, tmax = -1., 4.
event_id = dict(hands=2, feet=3)
subject = 1
runs = [6, 10, 14]  # motor imagery: hands vs feet

raw_fnames = eegbci.load_data(subject, runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
eegbci.standardize(raw)  # set channel names
montage = make_standard_montage('standard_1005')
raw.set_montage(montage)

# Apply band-pass filter
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
labels = epochs.events[:, -1] - 2

# %%
# Classification with linear discrimant analysis

# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=None)

# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

# plot CSP patterns estimated on full data for visualization
csp.fit_transform(epochs_data, labels)

csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

# %%
# Look at performance over time

sfreq = raw.info['sfreq']
w_length = int(sfreq * 0.5)   # running classifier: window length
w_step = int(sfreq * 0.1)  # running classifier: window step size
w_start = np.arange(0, epochs_data.shape[2] - w_length, w_step)

scores_windows = []

for train_idx, test_idx in cv_split:
    y_train, y_test = labels[train_idx], labels[test_idx]

    X_train = csp.fit_transform(epochs_data_train[train_idx], y_train)
    X_test = csp.transform(epochs_data_train[test_idx])

    # fit classifier
    lda.fit(X_train, y_train)

    # running classifier: test classifier on sliding window
    score_this_window = []
    for n in w_start:
        X_test = csp.transform(epochs_data[test_idx][:, :, n:(n + w_length)])
        score_this_window.append(lda.score(X_test, y_test))
    scores_windows.append(score_this_window)

# Plot scores over time
w_times = (w_start + w_length / 2.) / sfreq + epochs.tmin

plt.figure()
plt.plot(w_times, np.mean(scores_windows, 0), label='Score')
plt.axvline(0, linestyle='--', color='k', label='Onset')
plt.axhline(0.5, linestyle='-', color='k', label='Chance')
plt.xlabel('time (s)')
plt.ylabel('classification accuracy')
plt.title('Classification score over time')
plt.legend(loc='lower right')
plt.show()

##############################################################################
# References
# ----------
# .. footbibliography::

In [ ]:
mne.channels.get_builtin_montages()

In [ ]:
# chanPos["ch_pos"].keys()
montage = make_standard_montage("GSN-HydroCel-64_1.0")
chanPos = montage.get_positions()
list(chanPos["ch_pos"].keys())

In [ ]:
fname = "/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Raw/P03.xdf"
streams, header = pyxdf.load_xdf(fname)
data = streams[0]["time_series"].T
# assert data.shape[0] == 64  # four raw EEG plus one stim channel
# data[:4:2] -= data[1:4:2]  # subtract (rereference) to get two bipolar EEG
# data = data[::2]  # subselect
# data[:2] *= (1e-6 / 50 / 2)  # uV -> V and preamp gain
sfreq = float(streams[0]["info"]["nominal_srate"][0])
info = mne.create_info(64, sfreq, ["eeg"] * 64)
raw = mne.io.RawArray(data, info)

In [ ]:
data.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

print(__doc__)

# #############################################################################
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.
tmin, tmax = -1., 4.
event_id = dict(hands=2, feet=3)
subject = 1
runs = [6, 10, 14]  # motor imagery: hands vs feet

# raw_fnames = eegbci.load_data(subject, runs)
# raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
# eegbci.standardize(raw)  # set channel names
# montage = make_standard_montage('standard_1005')
# raw.set_montage(montage)

# raw = mne.io.read_raw_eeglab("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Raw/P03.xdf")
# montage = make_standard_montage("GSN-HydroCel-64_1.0")
# raw.set_montage(montage)

# Apply band-pass filter
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
labels = epochs.events[:, -1] - 2

In [ ]:
# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=None)

# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

# plot CSP patterns estimated on full data for visualization
csp.fit_transform(epochs_data, labels)

csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
epochs_data.shape, labels.shape

In [ ]:
sfreq = raw.info['sfreq']
w_length = int(sfreq * 0.5)   # running classifier: window length
w_step = int(sfreq * 0.1)  # running classifier: window step size
w_start = np.arange(0, epochs_data.shape[2] - w_length, w_step)

scores_windows = []

for train_idx, test_idx in cv_split:
    y_train, y_test = labels[train_idx], labels[test_idx]

    X_train = csp.fit_transform(epochs_data_train[train_idx], y_train)
    X_test = csp.transform(epochs_data_train[test_idx])

    # fit classifier
    lda.fit(X_train, y_train)

    # running classifier: test classifier on sliding window
    score_this_window = []
    for n in w_start:
        X_test = csp.transform(epochs_data[test_idx][:, :, n:(n + w_length)])
        score_this_window.append(lda.score(X_test, y_test))
    scores_windows.append(score_this_window)

# Plot scores over time
w_times = (w_start + w_length / 2.) / sfreq + epochs.tmin

plt.figure()
plt.plot(w_times, np.mean(scores_windows, 0), label='Score')
plt.axvline(0, linestyle='--', color='k', label='Onset')
plt.axhline(0.5, linestyle='-', color='k', label='Chance')
plt.xlabel('time (s)')
plt.ylabel('classification accuracy')
plt.title('Classification score over time')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# plot CSP patterns estimated on full data for visualization
csp.fit_transform(X, y)

csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)